In [1]:
P = Primes()
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m,x):
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m,x):
    f = J(n,m,x)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no14.txt','a') # xJ polynomials
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snipped

In [3]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)
print s[1]

201
[0, 3/8*x^2 + 1/2]


In [4]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=12
for k in [1..6]:
    index=s[k][0]
    poly = s[k][1]
    print poly

3/8*x^2 + 1/2
69/1024*x^4 - 1/128*x^2 - 3/64
1/128*x^6 - 29/864*x^4 + 1/216*x^2 + 1/54
5601/8388608*x^8 - 3821/524288*x^6 + 5195/262144*x^4 - 101/32768*x^2 - 303/32768
3/65536*x^10 - 16367/18432000*x^8 + 7961/1382400*x^6 - 21809/1728000*x^4 + 373/172800*x^2 + 373/72000
23003/8589934592*x^12 - 2254159/28991029248*x^10 + 338577733/391378894848*x^8 - 106154827/24461180928*x^6 + 68420351/8153726976*x^4 - 4754693/3057647616*x^2 - 4754693/1528823808


In [18]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=12
prime=3
for k in [1..6]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    print poly
    tf = splitsTF(poly,prime,1)
    print (index,tf)

3*x^2 + 4
(0, True)
69*x^4 - 8*x^2 - 48
(1, True)
27*x^6 - 116*x^4 + 16*x^2 + 64
(2, True)
5601*x^8 - 61136*x^6 + 166240*x^4 - 25856*x^2 - 77568
(3, True)
10125*x^10 - 196404*x^8 + 1273760*x^6 - 2791552*x^4 + 477440*x^2 + 1145856
(4, True)
16769187*x^12 - 486898344*x^10 + 5417243728*x^8 - 27175635712*x^6 + 52546829568*x^4 - 9737611264*x^2 - 19475222528
(5, True)


In [20]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=12
prime=3
no=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    tf = splitsTF(poly,prime,1)
    if tf==False:no=no+[index]
print no

[8, 11, 17, 23, 34, 38, 49, 50, 80, 83, 85, 104, 106, 116, 125, 136, 142, 160, 169, 178, 187, 188, 194, 196]


In [24]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
sd=1
for k in [0..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    tf = splitsTF(poly,prime,sd)
    if tf==True:
        td=totalDegreeOverField(poly,prime,sd)
        residue=ZZ(mod(index,prime))
        l = (index-residue)/prime
        print (index,l,td)
    
   

(-1, -1, 0)
(0, 0, 0)
(1, 0, 2)
(2, 0, 4)
(3, 1, 6)
(4, 1, 6)
(5, 1, 8)
(6, 2, 10)
(7, 2, 10)
(9, 3, 14)
(10, 3, 16)
(12, 4, 18)
(13, 4, 18)
(14, 4, 20)
(15, 5, 22)
(16, 5, 22)
(18, 6, 26)
(19, 6, 26)
(20, 6, 28)
(21, 7, 30)
(22, 7, 32)
(24, 8, 34)
(25, 8, 34)
(26, 8, 36)
(27, 9, 38)
(28, 9, 38)
(29, 9, 40)
(30, 10, 42)
(31, 10, 42)
(32, 10, 44)
(33, 11, 46)
(35, 11, 48)
(36, 12, 50)
(37, 12, 50)
(39, 13, 54)
(40, 13, 56)
(41, 13, 56)
(42, 14, 58)
(43, 14, 58)
(44, 14, 60)
(45, 15, 62)
(46, 15, 64)
(47, 15, 64)
(48, 16, 66)
(51, 17, 70)
(52, 17, 70)
(53, 17, 72)
(54, 18, 74)
(55, 18, 76)
(56, 18, 76)
(57, 19, 78)
(58, 19, 80)
(59, 19, 80)
(60, 20, 82)
(61, 20, 82)
(62, 20, 84)
(63, 21, 86)
(64, 21, 86)
(65, 21, 88)
(66, 22, 90)
(67, 22, 90)
(68, 22, 92)
(69, 23, 94)
(70, 23, 96)
(71, 23, 96)
(72, 24, 98)
(73, 24, 98)
(74, 24, 100)
(75, 25, 102)
(76, 25, 102)
(77, 25, 104)
(78, 26, 106)
(79, 26, 106)
(81, 27, 110)
(82, 27, 112)
(84, 28, 114)
(86, 28, 118)
(87, 29, 118)
(88, 29, 120)
(89

In [3]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if sd>0:data=data+[sd]
print Set(data)

{1, 2, 3, 4}


In [40]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
for k in [0..10]:
    index=s[k][0]
    residue=ZZ(mod(index,prime))
    if index!=residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l = (index-residue)/prime
        data = data + [(l,sd,td)]
print Set(data)

{(-1, 1, 0), (2, 3, 14), (3, 1, 14), (1, 1, 8), (1, 1, 6), (2, 1, 10)}


In [47]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
lset=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    residue=ZZ(mod(index,prime))
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    td=totalDegreeOverField(poly,prime,sd)
    l = (index-residue)/prime
    lset=lset+[l]
    data = data + [(l,sd)]
lset=Set(lset)
setdata=Set(data)
lsd=len(setdata)
print setdata
print lsd
print lset


{(7, 3), (12, 1), (47, 2), (45, 1), (41, 2), (38, 1), (62, 2), (15, 1), (48, 1), (25, 1), (27, 2), (51, 1), (18, 1), (44, 1), (21, 1), (16, 3), (47, 1), (28, 2), (14, 1), (38, 2), (57, 1), (24, 1), (59, 2), (1, 1), (50, 1), (27, 1), (53, 1), (20, 1), (46, 1), (2, 3), (23, 1), (56, 1), (33, 1), (0, 1), (59, 1), (45, 2), (26, 1), (3, 1), (52, 1), (64, 2), (29, 1), (34, 3), (12, 3), (55, 1), (22, 1), (65, 1), (32, 1), (9, 1), (58, 1), (11, 2), (35, 1), (30, 1), (2, 1), (26, 2), (61, 1), (56, 3), (65, 3), (28, 1), (5, 1), (54, 1), (31, 1), (64, 1), (-1, 1), (41, 1), (8, 1), (53, 2), (34, 1), (11, 1), (60, 1), (37, 1), (4, 1), (63, 1), (5, 4), (35, 3), (7, 1), (40, 1), (17, 1), (66, 1), (43, 1), (10, 1), (36, 1), (3, 3), (13, 1), (62, 1), (39, 1), (6, 1), (49, 1), (16, 1), (42, 1), (19, 1)}
90
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 5

In [48]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
lset=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    residue=ZZ(mod(index,prime))
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    td=totalDegreeOverField(poly,prime,sd)
    l = (index-residue)/prime
    lset=lset+[l]
    data = data + [(l,td)]
lset=Set(lset)
setdata=Set(data)
lsd=len(setdata)
print setdata
print lsd
print lset



{(60, 242), (62, 250), (-1, 0), (16, 68), (6, 28), (47, 192), (51, 206), (39, 158), (1, 6), (43, 174), (11, 48), (44, 180), (48, 196), (41, 166), (28, 118), (59, 240), (55, 224), (0, 4), (31, 126), (58, 236), (35, 142), (62, 256), (29, 118), (60, 244), (33, 138), (6, 26), (18, 76), (47, 190), (57, 230), (34, 138), (12, 50), (3, 18), (24, 98), (61, 248), (62, 254), (20, 82), (28, 120), (10, 42), (15, 62), (11, 52), (63, 258), (54, 220), (63, 254), (38, 158), (32, 130), (13, 54), (28, 114), (26, 110), (59, 244), (19, 80), (38, 156), (9, 40), (64, 262), (11, 46), (25, 104), (31, 128), (50, 202), (27, 110), (53, 214), (20, 84), (16, 66), (23, 96), (57, 234), (27, 112), (23, 94), (42, 170), (53, 220), (54, 218), (48, 194), (22, 92), (40, 164), (9, 38), (0, 2), (7, 34), (45, 182), (46, 188), (27, 114), (25, 102), (66, 266), (29, 120), (26, 106), (52, 210), (8, 36), (14, 60), (59, 238), (21, 86), (36, 146), (56, 228), (65, 264), (3, 16), (16, 70), (17, 72), (40, 162), (51, 208), (22, 90), (10

In [49]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
lset=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    residue=ZZ(mod(index,prime))
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    td=totalDegreeOverField(poly,prime,sd)
    l = (index-residue)/prime
    l=ord(3,l)
    lset=lset+[l]
    data = data + [(l,sd)]
lset=Set(lset)
setdata=Set(data)
lsd=len(setdata)
print setdata
print lsd
print lset



{(0, 1), (3, 2), (1, 3), (3, 1), (2, 1), (1, 1), (2, 2), (0, 4), (0, 3), (0, 2), (+Infinity, 1)}
11
{0, 1, 2, 3, +Infinity}


In [51]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
lset=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    residue=ZZ(mod(index,prime))
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    td=totalDegreeOverField(poly,prime,sd)
    l = (index-residue)/prime
    l=ord(3,l)
    lset=lset+[l]
    data = data + [(l,td)]
lset=Set(lset)
setdata=Set(data)
lsd=len(setdata)
print setdata
print lsd
print lset

{(0, 20), (0, 210), (1, 196), (3, 114), (0, 144), (0, 10), (1, 64), (0, 200), (0, 32), (1, 122), (0, 238), (2, 146), (0, 70), (1, 28), (1, 138), (1, 244), (0, 128), (1, 54), (0, 58), (1, 172), (0, 248), (0, 166), (0, 80), (3, 220), (+Infinity, 2), (0, 30), (0, 220), (1, 194), (0, 118), (0, 154), (0, 52), (0, 262), (0, 176), (0, 42), (2, 40), (1, 158), (0, 214), (1, 26), (0, 14), (2, 74), (1, 242), (0, 102), (0, 138), (0, 36), (1, 170), (0, 226), (2, 38), (0, 90), (+Infinity, 4), (1, 16), (0, 198), (0, 132), (2, 184), (1, 160), (0, 256), (0, 186), (0, 84), (0, 18), (2, 148), (3, 112), (0, 158), (0, 8), (0, 246), (0, 96), (0, 180), (0, 46), (1, 100), (0, 236), (2, 256), (0, 170), (0, 68), (1, 88), (0, 192), (0, 142), (0, 56), (0, 164), (0, 94), (3, 218), (+Infinity, 0), (0, 28), (1, 86), (0, 152), (1, 14), (0, 240), (1, 230), (0, 106), (0, 190), (3, 110), (1, 98), (0, 22), (0, 212), (0, 78), (0, 202), (0, 204), (1, 62), (0, 34), (0, 224), (0, 174), (1, 208), (0, 120), (1, 18), (0, 6), (0

In [27]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    residue=ZZ(mod(index,prime))
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    td=totalDegreeOverField(poly,prime,sd)
    l=(index-residue)/prime
    print (index,l,td)


(-1, -1, 0)
(0, 0, 0)
(1, 0, 2)
(2, 0, 4)
(3, 1, 6)
(4, 1, 6)
(5, 1, 8)
(6, 2, 10)
(7, 2, 10)
(8, 2, 14)
(9, 3, 14)
(10, 3, 16)
(11, 3, 18)
(12, 4, 18)
(13, 4, 18)
(14, 4, 20)
(15, 5, 22)
(16, 5, 22)
(17, 5, 28)
(18, 6, 26)
(19, 6, 26)
(20, 6, 28)
(21, 7, 30)
(22, 7, 32)
(23, 7, 34)
(24, 8, 34)
(25, 8, 34)
(26, 8, 36)
(27, 9, 38)
(28, 9, 38)
(29, 9, 40)
(30, 10, 42)
(31, 10, 42)
(32, 10, 44)
(33, 11, 46)
(34, 11, 52)
(35, 11, 48)
(36, 12, 50)
(37, 12, 50)
(38, 12, 54)
(39, 13, 54)
(40, 13, 56)
(41, 13, 56)
(42, 14, 58)
(43, 14, 58)
(44, 14, 60)
(45, 15, 62)
(46, 15, 64)
(47, 15, 64)
(48, 16, 66)
(49, 16, 68)
(50, 16, 70)
(51, 17, 70)
(52, 17, 70)
(53, 17, 72)
(54, 18, 74)
(55, 18, 76)
(56, 18, 76)
(57, 19, 78)
(58, 19, 80)
(59, 19, 80)
(60, 20, 82)
(61, 20, 82)
(62, 20, 84)
(63, 21, 86)
(64, 21, 86)
(65, 21, 88)
(66, 22, 90)
(67, 22, 90)
(68, 22, 92)
(69, 23, 94)
(70, 23, 96)
(71, 23, 96)
(72, 24, 98)
(73, 24, 98)
(74, 24, 100)
(75, 25, 102)
(76, 25, 102)
(77, 25, 104)
(78, 26, 106)
(7

In [28]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if mod(index,prime)==0:
        data=data+[sd]
setteddata=Set(data)
print len(setteddata);print setteddata

1
{1}


In [29]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if mod(index,prime)==1:
        data=data+[sd]
setteddata=Set(data)
print len(setteddata);print setteddata

3
{1, 2, 3}


In [30]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if mod(index,prime^2)==1:
        data=data+[sd]
setteddata=Set(data)
print len(setteddata);print setteddata

2
{1, 2}


In [32]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if mod(index,prime^4)==1:
        data=data+[sd]
setteddata=Set(data)
print len(setteddata);print setteddata

1
{1}


In [36]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    residue=ZZ(mod(index,prime))
    o3=ord(prime,index-residue)
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    print (residue,sd)
    

(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 3)
(0, 1)
(1, 1)
(2, 3)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 4)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 3)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 2)
(2, 1)
(0, 1)
(1, 1)
(2, 3)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 3)
(2, 3)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 2)
(0, 1)
(1, 1)
(2, 2)
(0, 1)
(1, 2)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 3)
(0, 1)
(1, 3)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 2)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 2)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)
(1, 2)
(2, 1)
(0, 1)
(1, 1)
(2, 1)
(0, 1)

In [37]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    residue=ZZ(mod(index,prime))
    o3=ord(prime,index-residue)
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    print (o3,sd)
    

(1, 1)
(+Infinity, 1)
(+Infinity, 1)
(+Infinity, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 3)
(2, 1)
(2, 1)
(2, 3)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 4)
(2, 1)
(2, 1)
(2, 1)
(1, 1)
(1, 1)
(1, 3)
(1, 1)
(1, 1)
(1, 1)
(3, 1)
(3, 1)
(3, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 2)
(1, 1)
(2, 1)
(2, 1)
(2, 3)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(2, 1)
(2, 1)
(2, 1)
(1, 1)
(1, 3)
(1, 3)
(1, 1)
(1, 1)
(1, 1)
(3, 1)
(3, 1)
(3, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(2, 1)
(2, 1)
(2, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(2, 1)
(2, 1)
(2, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 2)
(4, 1)
(4, 1)
(4, 2)
(1, 1)
(1, 2)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(2, 1)
(2, 1)
(2, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(2, 1)
(2, 1)
(2, 1)
(1, 1)
(1, 1)
(1, 3)
(1, 1)
(1, 3)
(1, 1)
(3, 1)
(3, 1)
(3, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 2)
(2, 1)
(2, 1)
(2, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 2)
(2, 1)
(2, 1)
(2, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(3, 1)
(3, 2)
(3, 1)
(1,

In [38]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    residue=ZZ(mod(index,prime))
    o3=ord(prime,index)
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    print (o3,sd)

(0, 1)
(+Infinity, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 3)
(2, 1)
(0, 1)
(0, 3)
(1, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 4)
(2, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 3)
(1, 1)
(0, 1)
(0, 1)
(3, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 2)
(0, 1)
(2, 1)
(0, 1)
(0, 3)
(1, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 1)
(2, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 3)
(0, 3)
(1, 1)
(0, 1)
(0, 1)
(3, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 1)
(2, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 1)
(2, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 2)
(4, 1)
(0, 1)
(0, 2)
(1, 1)
(0, 2)
(0, 1)
(1, 1)
(0, 1)
(0, 1)
(2, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 1)
(2, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 3)
(1, 1)
(0, 3)
(0, 1)
(3, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 2)
(2, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 2)
(2, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 1)
(3, 1)
(0, 2)
(0, 1)
(1, 1)
(0, 1)
(0, 1